### 1.**필요한 라이브러리, 모듈 로드**
* https://eu4ng.tistory.com/37

In [1]:
#  데이터 처리 모듈
import pandas as pd
from sklearn.impute import KNNImputer

# 행렬 등 통계 연산 모듈
import numpy as np

# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format

# 타입 어노테이션(Any, Sequencsss의 메서드 활용)
from typing import *
import os

# 정규표현식(Regex)
import re

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

# 조합 찾는 반복문 메서드
from itertools import permutations
from itertools import combinations

# 추상 클래스와 추상 메서드, 데코레이터 활용 모듈
from abc import ABC, abstractmethod

Mounted at /content/drive


### 2.**데이터 로드**

In [3]:
class GoogleDriveDataLoad:
  def __init__(self):
    self.file_path = None
    self.file_name = None

  def loadData(#self,
               file_path: str, file_name: str, encoder: str, id: int) -> pd.DataFrame:
    return pd.read_csv(
        os.path.join(file_path, file_name), encoding = encoder, index_col = id)

In [4]:
load = GoogleDriveDataLoad

In [5]:
DATA_LIST: Dict = {}
FILE_LIST = os.listdir("/content/drive/MyDrive/PseudonymizeData/Data Archive")

In [6]:
for index, file in zip(list(range(0, 7, 1)), FILE_LIST):
  data = load.loadData(
      file_path = "/content/drive/MyDrive/PseudonymizeData/Data Archive",
      file_name = file,
      encoder = "utf-8",
      id = 0)
  DATA_LIST[file] = data

In [ ]:
for index in range(0, 7, 1):
  print("\n", FILE_LIST[index])
  DATA_LIST[FILE_LIST[index]].info()

In [ ]:
DATA_LIST[FILE_LIST[0]]

### 3.**가명처리 클래스**

* 활용 통신 데이터 요구수준표 및 항목별 가명처리 계획서

| 연번 | 변수명 | 변수 설명 | 처리 방법 1 | 처리 수준 1 | 처리 방법 2 | 처리 수준 2 |
| --- | --- | --- | --- | --- | --- | --- |
| 0 | 일련번호 | 데이터 구별을 위한 순번 |  |  |  |  |
| 1 | 나이 | 20세 ~ 80세, 상세 나이 | 상하단 코딩 | 0.01 |  |  |
| 2 | 이름 | 이름 | 삭제 |  |  |  |
| 3 | 성별 | male / female | 원본유지 |  |  |  |
| 4 | 우편번호 | 5자릿수 우편번호 | 삭제 |  |  |  |
| 5 | 상세주소 | 상세주소 | 범주화 | 읍면동 |  |  |
| 6 | 휴대폰 번호 | 핸드폰 번호 | 삭제 |  |  |  |
| 7 | 직업 | 표준산업분류 대분류 이름 | 원본유지 |  |  |  |
| 8 | 가입상품 | SKT 인터넷 상품 | 원본유지 |  |  |  |
| 9 | 요금 | SKT 인터넷 상품요금 | 상하단 코딩 | 0.01 |  |  |
| 10 | 가입일 | 2018 ~ 2021년, 5년 동안 가입한 고객 | 범주화 | 연월 |  |  |
| 11 | 회원유형 | 홀, 모바일, 개인사업자, 법인으로 구분 | 원본유지 |  |  |  |
| 12 | 블랙리스트 유형 | 악성 클레임 고객 여부 | 삭제 |  |  |  |
| 13 | 연체 횟수 | 5년간 요금 연체 횟수 | 상하단 코딩 | 0.01 |  |  |
| 14 | 멤버십 등급 | VVIP, VIP, GOLD, SILVER, WHITE, 일반으로 구분 | 원본유지 |  |  |  |
| 15 | 콘텐츠 구매 여부 | 인터넷 콘텐츠 구매 여부 | 원본유지 |  |  |  |
| 16 | 구매금액 | 인터넷 콘텐츠 구매 고객의 총 구매금액 | 라운딩 | 3 | 상하단 코딩 | 0.01 |
| 17 | 부가 서비스 | 악성코드 차단, 가디언, 세이퍼플러스, 원스톱, 미가입으로 구분 | 원본유지 |  |  |  |

### 4.구현의 핵심 목표는 **가명처리 패키지를 구조화하고 필요한 기능을 제공하기 위해 디렉토리 구조와 객체 및 속성을 설계**함에 있다.

In [9]:
class Pseudonymizer(ABC):
    """가명처리 추상 클래스 및 추상 메서드 선언"""
    @abstractmethod
    def pseudonymizeData(self, value):
        """확장성을 갖춘 가명처리 클래스를 만들어 특정 가명처리 기법으로 구체화하기 위한 추상 메서드"""
        pass

In [10]:
DATA_LIST[FILE_LIST[0]].head(3)

,이름,성별,나이,핸드폰번호,우편번호,집주소,주거유형,소득분위,신용평가모델점수,상환위험지수,신용카드월사용금액,신용카드월현금서비스금액,신용카드발급수,해지카드수,대출여부,대출총금액,미상환대출금액,연금수급여부,연금수급액(월)
0,강현심,female,80,010-1826-4535,27218,충청북도 제천시 수산면 옥순봉로6길 61-32,"판잣집, 비닐하우스, 컨테이너, 움막",6,441,61,5357418,8583708,3,5,N,0,0,N,0
1,서삼봉,female,30,010-1765-1467,12736,경기도 광주시 초월읍 도곡길 109-9,다세대주택,8,515,10,8870904,5328233,3,1,N,0,0,N,0
2,팽상아,female,53,010-6593-5091,51292,경상남도 창원시 마산회원구 회원남32길 28-1(회원동),"판잣집, 비닐하우스, 컨테이너, 움막",1,269,17,8525658,2005663,6,5,N,0,0,N,0


### 5.**MaskingPseudonymizer | 마스킹 적용 사례(정규표현식을 주로 활용)**

* 성명: 직접 식별자에 대해 마스킹을 적용하여 식별성을 낮출 수 있음. 단, 이름이 세글자가 아닌 경우에는 어떻게 할지(두글자이거나 네글자 이상일 경우. 한국인이 아닌 외국인일 경우 어떻게 처리할지)

* 나이 : 간접 식별자에 대해 마스킹을 적용할 수는 있지만 마스킹 방식을 거의 사용하지 않음

* 질병코드 : 민감 질병에 대해 마스킹을 적용하여 식별성을 낮춘 사례

* 사업자 등록번호 :

    (1) 법인의 종류를 알 수 없도록 가운데 마스킹(2자릿수)
    
    (2) 법인의 종류만 구분가능하도록 맨 마지막 마스킹(5자릿수)
    
* 이메일 주소 : 메일발신기관만 구분가능하도록 마스킹하거나 발신자와 발신기관 모두 구분할 수 없도록 마스킹

* 특수문자 체크 정규식
  
  const regExp = /[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]/g;

* 모든 공백 체크 정규식
  
  const regExp = /\s/g;

* 숫자만 체크 정규식

  const regExp = /[0-9]/g;

* 이메일 체크 정규식
  
  const regExp = /^[0-9a-zA-Z]([-_\.]?[0-9a-zA-Z])*@[0-9a-zA-Z]([-_\.]?[0-9a-zA-Z])*\.[a-zA-Z]{2,3}$/i;

* 핸드폰번호 정규식
  
  const regExp = /^\d{3}-\d{3,4}-\d{4}$/;

* 일반 전화번호 정규식
  
  const regExp = /^\d{2,3}-\d{3,4}-\d{4}$/;

* 아이디나 비밀번호 정규식
  
  const regExp = /^[a-z0-9_]{4,20}$/;

* 휴대폰번호 체크 정규식
  
  const regExp = /^01([0|1|6|7|8|9]?)-?([0-9]{3,4})-?([0-9]{4})$/;

In [11]:
# ./pseudonymizer/pseudonymizers/
# from pseudonymizer.pseudonymizer import Pseudonymizer
class NameMaskingModule(Pseudonymizer):
    """
    성명 마스킹 클래스
    ------------------
    성명 식별자에 직접 마스킹을 적용하여 식별성을 낮추는 구체 클래스
    3자릿수가 넘는 이름은 특정인에 대한 식별가능성이 높아지므로 4자릿수로 마스킹

    자식 클래스 메서드 오버라이딩
    -----------------------------
    구체 상속 클래스 선언 시 @property와 @psuedonymizeData.setter 데코레이터로 추상화한다.
    개인식별정보 또는 민감정보 마스킹 시 그 변수 값을 외부에서 접근하지 못하도록 막기 위한 의도
    """
    def pseudonymizeData(self, name):
        name_list = list(name)
        if 2 <= len(name) <= 3:
            return name_list[0] + "*"*(len(name)-1)
        else:
            # len(name) > 4:
            return name_list[0] + "*"*3

In [12]:
# from pseudonymizer.pseudonymizer import Pseudonymizer
# import re
# from typing import *

class EmailMaskingModule(Pseudonymizer):
    """
    이메일 마스킹 클래스
    --------------------
    이메일 주소의 메일 발신자 또는 발신기관을 구분할 수 없도록 하는 구체 클래스
    """
    def __init__(self, masking_domain: bool):
        self.masking_domain = masking_domain

    def pseudonymizeData(self, email):
        pattern_match = re.match(
            r"^[a-zA-z0-9]([-_\.]?[a-zA-Z0-9])*@[a-zA-Z0-9]+\(.[a-zA-Z]{2,3})", email)
            # .: 정확히 1개 문자 매칭
            # * : 앞 패턴이 0개 이상이어야 함
            # ? : 앞 패턴이 없거나 하나이어야 함
            # + : 1회 이상 반복되는 패턴을 매칭
            # \. : 도메인과 최상위 도메인(TLD)에 대한 구분자 마침표
            # {,} : 중괄호 안에 표기된 범위만큼 반복되는 패턴을 매칭. {3,5}는 3~5회 매칭을 의미함

        if pattern_match:
            local_part = pattern_match.group(0)
            domain_part = pattern_match.group(1)
            tld_part = pattern_match.group(2)
            # local_part, domain_part = email.split("@")

            if self.masking_domain:
                masked_local_part = re.sub(r"\S", "*", local_part)
                masked_domain_part = re.sub(r"\S", "*", domain_part)
                return masked_local_part + "@" + masked_domain_part + tld_part
            else:
              # self.masking_domain = False:
                masked_local_part = re.sub(r"\S", "*", local_part)
                return masked_local_part + "@" + domain_part + tld_part

        else:
            print("입력받은 { }은 이메일 패턴에 매칭되지 않아 마스킹할 수 없습니다.".format(email))

        # 이메일의 표준은 인터넷 표준 기구(IETF, Internet Engineering Task Force)에서 정의
        # re.sub(pattern, replace, text)

In [14]:
# from pseudonymizer.pseudonymizer import Pseudonymizer

class ResidentNumberMaskingModule(Pseudonymizer):
    """
    주민등록번호 마스킹 클래스
    --------------------
    주민등록번호 뒷자리를 복원할 수 없는 비가역성 기법으로 개인의 식별을 방지하는 구체 클래스
    """
    def pseudonymizeData(self, resident_number):
        """한국 주민등록번호의 정규표현식을 기준으로 패턴 매칭이 되는 경우 뒷자리 7자리 중
        성별정보 1자리를 제외한 나머지 출생지정보에 대한 6자리에 대한 마스킹을 수행하는 메서드"""
        pattern_match = re.match(r"^\d{6}-[1-8]\d{6}$", resident_number)

        if pattern_match:
            front_part, rear_part = resident_number.split("-")
            return front_part + rear_part[0] + "*"*6

        else:
            print("입력받은 { }은 주민등록번호 패턴에 매칭되지 않아 마스킹할 수 없습니다.".format(resident_number))

```
class PartialSuppressingPseudonymizer(Pseudonymizer):
  """부분 삭제 클래스: 삭제[부분, 로컬, 행, 열]"""
  def __init__(self):

  def pseudonymizeData(self, value):
    """주소의 상세 지번 정보를 삭제하거나 날짜 형식에서 연-월-일을 연-월만 남기고 일자를 삭제하는 메서드"""
```

In [15]:
# from pseudonymizer.pseudonymizer import Pseudonymizer
# from typing import *

class BusinessNumberMaskingModule(Pseudonymizer):
    """
    사업자등록번호 마스킹 클래스
    --------------------
    사업자등록번호의 일부(2번째 혹은 3번째 자리)를 복원할 수 없는 비가역성 기법으로 개인의 식별을 방지하는 구체 클래스
    """
    def __init__(self, masking_part: str):
        self.masking_part = masking_part

    def pseudonymizeData(self, business_number):
        """한국 사업자등록번호의 정규표현식을 기준으로 패턴 매칭이 되는 경우 마스킹을 수행하는 메서드"""
        pattern_match = re.match(r"^\d{3}-\d{2}-\d{5}$", business_number)
        # ex) 124-86-23875

        if pattern_match:
            front_part, middle_part, rear_part = business_number.split("-")

           # 2번째 자리인 법인의 등록지역을 마스킹할 때
            if self.masking_part == "middle":
                return front_part + "*"*2 + rear_part
           # 3번째 자리인 법인의 일련번호를 마스킹할 때
            elif self.masking_part == "rear":
                return front_part + middle_part + "*"*5
           # 1번째 자리인 법인의 유형만을 남기고 마스킹할 때
            elif self.masking_part == "both":
                return front_part + "*"*2 + "-" + "*"*5
        else:
            print("입력받은 { }은 사업자등록번호 패턴에 매칭되지 않아 마스킹할 수 없습니다.".format(business_number))

In [16]:
# from pseudonymizer.pseudonymizer import Pseudonymizer
# from typing import *

class PhoneNumberMaskingModule(Pseudonymizer):
    """
    연락처(휴대전화번호 혹은 일반전화번호) 마스킹 클래스
    --------------------
    3번째 자리를 복원할 수 없는 비가역성 기법으로 개인의 식별을 방지하는 구체 클래스
    특히 연관된 다른 정보(생년월일, 기념일, 가족 전화번호, 기존 통화내역 등)와 쉽게 결합하여 사용자가 누구인지 식별가능하다는 점에서 개인정보에 해당함
    """
    def pseudonymizeData(self, phone_number):
        """전화번호의 정규표현식을 기준으로 패턴 매칭이 되는 경우 마스킹을 수행하는 메서드"""
        pattern_match = re.match(r"^\[0-9]-\[0-9]-\d{4}$", phone_number)

        if pattern_match:
            front_part, middle_part, rear_part = phone_number.split("-")
            return front_part + middle_part + "*"*4
        # 다만, 전화번호 마지막 4자리는 ****와 같은 기호로 대체하지 않고 전체를 해시값으로 암호화하기도 한다는 점에 유의하여야 함
        # 암호화와 복호화는 알고리즘 및 키 관리 등 복잡한 과정이 필요하므로 높은 보안이 필요한 경우 고려함
        else:
            print("입력받은 { }은 전화번호 패턴에 매칭되지 않아 마스킹할 수 없습니다.".format(phone_number))

In [20]:
# from pseudonymizer.pseudonymizers.nameMasking import NameMaskingModule
# from pseudonymizer.pseudonymizers.emailMasking import EmailMaskingModule
# from pseudonymizer.pseudonymizers.residentNumMasking import ResidentNumberMaskingModule
# from pseudonymizer.pseudonymizers.businessNumMasking import BusinessNumberMaskingModule
# from pseudonymizer.pseudonymizers.phoneNumMasking import PhoneNumberMaskingModule

# import re
# from typing import *

class MaskingPseudonymizer(Pseudonymizer):
    def __init__(self, data_type: str, masking_domain: bool, masking_part: str):
        """data_type은 향후 pseudonymizer.py에서 Pseudonymn 실행 클래스의
        self._dataframe[column] 개인식별정보의 유형으로 이름, 이메일, 주민등록번호, 사업자등록번호 중 하나로 선언"""
        self.data_type = data_type
        self.email_masker = EmailMaskingModule(masking_domain)
        self.name_masker = NameMaskingModule()
        self.resident_num_masker = ResidentNumberMaskingModule()
        self.business_num_masker = BusinessNumberMaskingModule(masking_part)
        self.phone_num_masker = PhoneNumberMaskingModule()

    def pseudonymizeData(self, data):
        if self.data_type == "name":
            return self.name_masker.pseudonymzieData(data)
        elif self.data_type == "email":
            return self.email_masker.pseudonymzieData(data)
        elif self.data_type == "resident_number":
            return self.resident_num_masker.pseudonymizeData(data)
        elif self.data_type == "business_number":
            return self.business_num_masker.pseudonymizeData(data)
        elif self.data_type == "phone_number":
            return self.phone_num_masker.pseudonymizeData(data)
        else:
            raise ValueError("유효한 마스킹 대상 개인식별정보 데이터 타입이 아닙니다.")

### 6.**RoundingPseudonymizer**
* 일반라운딩은 np.round()함수로 처리가능하므로 구현의 의미 없으므로 제외하며,
* 제어라운딩은 라운딩 적용 전후의 항목 합계를 일치시키면서 소요되는 높은 연산량으로 실무에서 활용하지 않으므로 제외
* 랜덤라운딩만 구현할 예정
* 나이, 신장, 소득, 카드지출금액, 유동인구, 사용자 수 등에 적용

In [22]:
# from pseudonymizer.pseudonymizer import Pseudonymizer
# from typing import *

class RandomRoundingPseudonymizer(Pseudonymizer):
    """
    랜덤라운딩 구체 클래스
    ----------------------
    데이터의 길이가 일정하지 않은 경우 값의 크기에 따라 처리 단위를 다르게 올림, 내림, 반올림하는 가명처리 기법
    """
    def __init__(self, rounding_type):
        self.rounding_type = rounding_type

    def pseudonymizeData(self, numeric):
        """수치데이터를 실제 수 기준으로 자릿수 올림 또는 내림하여 일반화(범주화)하는 메서드"""
        if self.rounding_type == "round_up":
            return numeric if numeric == int(numeric) else int(numeric)+1
        elif self.rounding_type == "round_down":
            return int(numeric)
        elif self.rounding_type == "round":
            decimal_part = numeric - int(numeric)
            return int(numeric)+1 if decimal_part >= 0.5 else int(numeric)
        else:
            raise ValueError("입력받은 {}은 유효한 라운딩 방법이 아닙니다.".format(self.rounding_type))

In [ ]:
class TopAndBottomCodingPseudonymizer(Pseudonymizer):
  """상하단코딩 클래스"""

  # def __init__(self, ):

  def pseudonymizeData(self, value):
    """적은 수의 분포를 가진 양 끝단의 정보를 범주화 등의 기법을 적용하여 식별성을 낮추는 메서드"""
    # if value is
    return value
    # valuevalue는 데이터프레임 람다 함수 내 익명(적적용)함수를 통해통해서 들어가므로 분포를 고려하는 방식을 적용하는게 쉽지는 않음

In [ ]:
class CategorizationPseudonymizer(Pseudonymizer):
  """범주화 클래스"""

  def __init__(self, categories):
    self.categories = categories

  def pseudonymizeData(self, value, **kwargs):
    """범주화 로직 구현 메서드(범주화된 값으로 대체하거나 새로운 범주로 매핑)"""
    # pd.cut외에 다른 방법은? 구간별로 분포를 나누고 범주를 할당하기 위해서는
    # if type(value) is str:
      #
    # elif type(value) is integer:
      #
    # elif type(value) is float:
      #
    # else:
      # 해당 변수의 자료형 자체를 범주화 클래스의 가명처리 인스턴스 적용 후에는, astype()로 범주형 변환
    return value

In [ ]:
class HashingPseudonymizer(Pseudonymizer):
  """해시 알고리즘 SHA256 등을 활용한 암호화(추가정보) 생성 클래스"""

  def pseudonymizeData(self, value, encoder):
    """sha256 알고리즘 기반의 해시함수 모듈로 해시값을 생성하되, 받은 문자열 합 일련번호에 솔트값을 추가하여 결합키 생성 메서드
    문제
    ----
    1. 동일한 메시지는 동일한 다이제스트를 갖는다(레인보우테이블 공격으로 다이제스트의 원문을 쉽게 얻을 수 있다)
    2. 생일 역설에 의한 해시충돌 브루스포스(무차별 대입 공격)

    대안
    ----
    crypt 모듈 — 유닉스 비밀번호 확인 함수

    """
    import hashlib
    value.split
    return hashlib.sha256( str(value).encode(encoder) ).hexdigest()

### 7.**동질 집합(Equivalent class) 찾기와 프라이버시 보호 모델에 따른 PPDM(Privacy Preserving Data Mining)**

In [23]:
class PrivacyPreservingModel:
    """개인식별가능정보 속성을 기준으로 데이터를 그룹화하는 클래스"""
    def __init__(self, dataframe):
        self._dataframe = dataframe
        self.equivalent_class = []

    def __str__(self):
        """주어진 데이터 셋의 컬럼 정보를 반환하는 메서드"""
        return self._dataframe.info()

    def categorizeEquivalentClass(self, attributes: List):
        # -> Dict[str, List[str]]:
        """각 행(레코드)에 대한 개인식별가능정보 속성(컬럼)들 사이에 동질 집합 확인"""
        # groupby_data = self._dataframe.groupby(attribute)
        # for group, data in groupby_data:

        groupby_data = self._dataframe[attributes].groupby(attributes)
            # self._dataframe[attributes].groupby( list(range(len(self._dataframe[attributes]))) )

        for _, group in groupby_data:
            # print(group)
            if len(group) > 1:
                self.equivalent_class.append(
                    group.index.tolist())
            # print(self.equivalent_class)
            # break

        return group, self.equivalent_class

    # def K_Anonymity(self):
       # """개별 레코드가 최소한 K개 이상의 동일한 속성값을 가지도록 하는 K-익명성 메서드"""

    # def L_Diversity(self):
       # """각 동질집합 내 특정 민감 속성의 빈도가 L값 이상의 다양성을 가지도록 하는 L-다양성 메서드"""

    # def T_Closeness(self):
       # """민감 정보(SA)의 분포를 전체 데이터 셋의 분포와 유사하도록 하는 T-근접성메서드"""

### 8.**가명처리 실행 클래스**

In [24]:
# ./pseudonymizer/pseudonymizer.py
# from abc import ABC, ABCMeta, abstractmethod
# import pandas as pd

class Pseudonym:
    def __init__(self, dataframe):
        """원본정보(재현데이터)와 가명처리 구체 클래스를 인스턴스 변수로 선언하는(초기화) 생성자"""
        self._dataframe = dataframe
        self._pseudonymizers = []
        self._pseudonymDictionary = {}

    def addPseudonymizer(self, pseudonymizer):
        """가명처리 추상 클래스에 대한 자식 클래스를 입력받는 pseudonymizer파라미터를 가지는 메서드"""
        if isinstance(pseudonymizer, Pseudonymizer):
            self._pseudonymizers.append(pseudonymizer)
        else:
            print("입력받은 {} 기술은 가명처리 기법에 추가할 수 없습니다.".format(pseudonymizer))

    def addDictionary(self, column, pseudonymizers):
        """가명처리를 수행할 데이터 컬럼명과 해당 열에 적용할 여러 가명처리 기법 리스트를 입력받아 다양한 비식별 조치를 수행할 수 있도록 지정하는 메서드"""
        self._pseudonymDictionary[column] = pseudonymizers

    def pseudonymizeData(self):
        """가명처리 기법을 해당 컬럼에 적용하는 메서드(apply함수를 활용하여 데이터프레임 모든 행, 특정 열에 비식별조치를 취하는 접근방식) """
        for column, pseudonymizers in self._pseudonymDictionary.items():
            for pseudonymizer in pseudonymizers:
                self._dataframe[column] = self._dataframe[column].apply(pseudonymizer.pseudonymizeData)

    def getPseudonymizedDataframe(self):
        """가명처리 데이터 반환"""
        return self._dataframe

In [25]:
# PseudonymizeFinanceData = Pseudonym(dataframe = DATA_FINANCE)